# WaveNet Scale: 10 Seeds + Build Submissions
**Goal**: Scale WaveNet-lite to 10 seeds, build canary + ONNX ensemble submissions.

**Strategy** (Codex-approved):
1. Train seeds 42-51
2. Build single best-val canary submission first
3. Build 10-model ONNX ensemble
4. Submit canary to measure val-LB gap before using ensemble slot

**ONNX**: Packs all ring buffers into single (1, 64, 1023) tensor for fast inference.
GRU ONNX was ~2.5x faster than PyTorch. Expect similar speedup for WaveNet.

**Pipeline**: Setup -> Train 10 seeds -> Evaluate -> Build ONNX zips -> Save

In [ ]:
# Cell 1: Mount Drive, download data from Kaggle, clone repo
import os, json, subprocess

from google.colab import drive
drive.mount('/content/drive')
os.makedirs('/content/drive/MyDrive/wunderfund', exist_ok=True)

!pip install -q kaggle==1.6.14 --force-reinstall
os.makedirs('/root/.kaggle', exist_ok=True)
with open('/root/.kaggle/kaggle.json', 'w') as f:
    json.dump({"username": "vincentvdo6", "key": "FILL_IN"}, f)
os.chmod('/root/.kaggle/kaggle.json', 0o600)

os.makedirs('/content/data', exist_ok=True)
!kaggle datasets download -d vincentvdo6/wunderfund-predictorium -p /content/data/ --force
!unzip -o -q /content/data/wunderfund-predictorium.zip -d /content/data/
!ls /content/data/*.parquet

# Clone repo
REPO = "/content/competition_package"
os.chdir("/content")
subprocess.run(["rm", "-rf", REPO], check=False)
subprocess.run(["git", "clone", "https://github.com/vincentvdo6/competition_package.git", REPO], check=True)
os.chdir(REPO)
os.makedirs("datasets", exist_ok=True)
os.makedirs("logs", exist_ok=True)

subprocess.run(["ln", "-sf", "/content/data/train.parquet", "datasets/train.parquet"], check=True)
subprocess.run(["ln", "-sf", "/content/data/valid.parquet", "datasets/valid.parquet"], check=True)

assert os.path.exists("datasets/train.parquet"), "train.parquet not found!"
assert os.path.exists("datasets/valid.parquet"), "valid.parquet not found!"

commit = subprocess.check_output(["git", "rev-parse", "--short", "HEAD"], text=True).strip()
print(f"Commit: {commit}")
print(f"GPU: {subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv,noheader'], text=True).strip()}")
print("Ready!")

In [ ]:
# Cell 2: Check for any existing checkpoints on Drive
import os, shutil
os.chdir("/content/competition_package")

DRIVE_DIR = "/content/drive/MyDrive/wunderfund"
restored = 0
for seed in range(42, 52):
    src = f"{DRIVE_DIR}/wavenet_v1_seed{seed}.pt"
    dst = f"logs/wavenet_v1_seed{seed}.pt"
    if os.path.exists(src) and not os.path.exists(dst):
        shutil.copy2(src, dst)
        restored += 1
        print(f"Restored: wavenet_v1_seed{seed}.pt")

if restored == 0:
    print("No WaveNet checkpoints found on Drive -- will train all 10 seeds")
else:
    print(f"\nRestored {restored} checkpoints from Drive")

In [ ]:
# Cell 3: Train WaveNet seeds 42-51 (skip existing)
import subprocess, sys, os
os.chdir("/content/competition_package")

SEEDS = list(range(42, 52))  # 42-51
CONFIG = "configs/wavenet_v1.yaml"

print(f"=== WAVENET SCALE ({len(SEEDS)} seeds) ===")
print(f"Config: {CONFIG}")
print("=" * 60, flush=True)

for seed in SEEDS:
    ckpt = f"logs/wavenet_v1_seed{seed}.pt"
    if os.path.exists(ckpt):
        print(f"seed {seed}: checkpoint exists -- skip")
        continue
    print(f"\n{'='*60}")
    print(f"WAVENET seed {seed}")
    print(f"{'='*60}", flush=True)
    proc = subprocess.Popen(
        [sys.executable, "-u", "scripts/train.py",
         "--config", CONFIG,
         "--seed", str(seed), "--device", "cuda"],
        stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
    )
    for line in proc.stdout:
        print(line, end="", flush=True)
    rc = proc.wait()
    if rc != 0:
        print(f"ERROR: seed {seed} failed with rc={rc}")

print(f"\nTraining complete!")

In [ ]:
# Cell 4: Evaluate all seeds
import os, torch
os.chdir("/content/competition_package")

SEEDS = list(range(42, 52))

print(f"{'Seed':<6} {'Val Score':>10} {'Epoch':>6}")
print("-" * 28)

results = []
for seed in SEEDS:
    pt = f"logs/wavenet_v1_seed{seed}.pt"
    if not os.path.exists(pt):
        print(f"s{seed}:  MISSING")
        continue
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    score = float(ckpt.get("best_score", 0))
    epoch = ckpt.get("best_epoch", "?")
    results.append((seed, score, epoch))
    print(f"s{seed:<5} {score:>10.4f} {epoch:>6}")

if results:
    scores = [r[1] for r in results]
    mean_val = sum(scores) / len(scores)
    best_seed, best_val, best_ep = max(results, key=lambda r: r[1])
    print(f"\nMean val ({len(results)} seeds): {mean_val:.4f}")
    print(f"Best seed: s{best_seed} = {best_val:.4f} (ep {best_ep})")
    print(f"GRU 10-seed mean val:      0.2708")
    print(f"GRU 10-seed LB:            0.2885")
    print(f"\nIf WaveNet has similar +0.018 gap: est LB = {mean_val + 0.018:.4f}")

In [ ]:
# Cell 5: Build ONNX submission zips (canary + ensemble)
import subprocess, sys, os, glob
os.chdir("/content/competition_package")

# Find all wavenet checkpoints
ckpts = sorted(glob.glob("logs/wavenet_v1_seed*.pt"))
ckpts = [c for c in ckpts if '_epoch' not in c]
print(f"Found {len(ckpts)} WaveNet checkpoints")

# Find best seed for canary
import torch
best_ckpt = None
best_score = -1
for c in ckpts:
    ck = torch.load(c, map_location="cpu", weights_only=False)
    s = float(ck.get("best_score", 0))
    if s > best_score:
        best_score = s
        best_ckpt = c
print(f"Best single: {os.path.basename(best_ckpt)} = {best_score:.4f}")

os.makedirs("submissions/ready", exist_ok=True)

# Build ONNX canary (single best)
print("\n=== Building ONNX canary (single best) ===")
proc = subprocess.Popen(
    [sys.executable, "scripts/build_wavenet_submission.py",
     "--checkpoints", best_ckpt, "--onnx",
     "--output", "submissions/ready/wavenet_canary_onnx.zip"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)
for line in proc.stdout:
    print(line, end="", flush=True)
proc.wait()

# Build ONNX ensemble (all seeds)
n = len(ckpts)
print(f"\n=== Building ONNX ensemble ({n} models) ===")
proc = subprocess.Popen(
    [sys.executable, "scripts/build_wavenet_submission.py",
     "--checkpoints"] + ckpts +
    ["--onnx", "--output", f"submissions/ready/wavenet_ens{n}_onnx.zip"],
    stdout=subprocess.PIPE, stderr=subprocess.STDOUT, text=True
)
for line in proc.stdout:
    print(line, end="", flush=True)
proc.wait()

# Show zip sizes
print("\n=== Submission zips ===")
for z in sorted(glob.glob("submissions/ready/wavenet*.zip")):
    sz = os.path.getsize(z) / 1e6
    print(f"  {os.path.basename(z)}: {sz:.1f}MB")

In [ ]:
# Cell 6: Save checkpoints + zips to Drive
import os, torch, shutil, glob
os.chdir("/content/competition_package")

DRIVE_DIR = "/content/drive/MyDrive/wunderfund"

# Save checkpoints (slim)
for pt in sorted(glob.glob("logs/wavenet_v1_seed*.pt")):
    if '_epoch' in pt:
        continue
    seed = os.path.basename(pt).replace('.pt', '').split('seed')[-1]
    ckpt = torch.load(pt, map_location="cpu", weights_only=False)
    slim = {
        "model_state_dict": ckpt["model_state_dict"],
        "config": ckpt.get("config", {}),
        "best_score": ckpt.get("best_score", None),
        "best_epoch": ckpt.get("best_epoch", None),
    }
    dst = f"{DRIVE_DIR}/wavenet_v1_seed{seed}.pt"
    torch.save(slim, dst)
    sz = os.path.getsize(dst) / 1e6
    print(f"Saved: wavenet_v1_seed{seed}.pt ({sz:.1f}MB)")

# Save zips
for z in glob.glob("submissions/ready/wavenet*.zip"):
    dst = f"{DRIVE_DIR}/{os.path.basename(z)}"
    shutil.copy2(z, dst)
    sz = os.path.getsize(dst) / 1e6
    print(f"Saved: {os.path.basename(z)} ({sz:.1f}MB)")

print("\nDone! Download zips for submission.")